In [1]:
import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from tensorflow.keras.datasets import imdb 
from sklearn.metrics import classification_report

In [2]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras import backend as K
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Bidirectional, Embedding, Dropout, Flatten, Layer, Input
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [3]:
from tensorflow.keras import models,layers,activations,losses,optimizers,metrics

In [4]:
n_unique_words = 10000
(x_train, y_train),(x_test, y_test) = imdb.load_data(num_words=n_unique_words)

In [5]:
maxlen = 200
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [67]:
#pure bilstm
model = Sequential()
model.add(Embedding(n_unique_words, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 200, 128)          1280000   
                                                                 
 bidirectional_12 (Bidirecti  (None, 128)              98816     
 onal)                                                           
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________


In [69]:
#model.fit(x_train, y_train, validation_split = 0.2,epochs = 12, verbose = True,batch_size = 64)

In [6]:
class Attention(Layer):
    def __init__(self,**kwargs):
        super(Attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(Attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(Attention,self).get_config()

In [40]:
inp = Input(shape=(200, ))
x = Embedding(10000, 128, trainable=False)(inp)
conv_1 = Conv1D(32, 9, activation='relu', name='conv1d_1')(x)
maxpool_1 = MaxPooling1D(16, name='maxpool1d_1')(conv_1)
dropout = Dropout(0.2, name='dropout_1')(maxpool_1)
#conv_2 = Conv1D(32, 7, activation='relu', name='conv1d_2')(dropout_1)
#maxpool_2 = MaxPooling1D(8,padding='same', name='maxpool1d_2')(conv_2)
#dropout_2 = Dropout(0.2, name='dropout_2')(maxpool_2)
x = Bidirectional(LSTM(64, return_sequences=True, dropout=0.25,recurrent_dropout=0.25))(dropout)
x = Attention()(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 200)]             0         
                                                                 
 embedding_2 (Embedding)     (None, 200, 128)          1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 192, 32)           36896     
                                                                 
 maxpool1d_1 (MaxPooling1D)  (None, 12, 32)            0         
                                                                 
 dropout_1 (Dropout)         (None, 12, 32)            0         
                                                                 
 bidirectional_2 (Bidirectio  (None, 12, 128)          49664     
 nal)                                                            
                                                           

In [41]:
def callbacks(**kwargs):
    mc = ModelCheckpoint(filepath = kwargs.get("filename"), save_best_only = True, verbose = 0)
    es = EarlyStopping(monitor = kwargs.get("monitor"), patience = kwargs.get("patience"))
    return mc,es

In [42]:
mc,es = callbacks(filename = "./cnn_bilstm.h5", patience = 3, monitor = "val_loss")

In [43]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#file_path = 'model.hdf5'
#ckpt = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
#early = EarlyStopping(monitor="val_loss", mode="min", patience=1)

In [44]:
model.fit(x_train, y_train, batch_size=256, epochs=20, validation_split=0.2,callbacks=[mc,es])

Epoch 1/20
79/79 [==============================] - 22s 195ms/step - loss: 0.6845 - accuracy: 0.5492 - val_loss: 0.6235 - val_accuracy: 0.6514
Epoch 2/20
79/79 [==============================] - 16s 197ms/step - loss: 0.5924 - accuracy: 0.6909 - val_loss: 0.5451 - val_accuracy: 0.7296
Epoch 3/20
79/79 [==============================] - 16s 198ms/step - loss: 0.5243 - accuracy: 0.7452 - val_loss: 0.5292 - val_accuracy: 0.7390
Epoch 4/20
79/79 [==============================] - 16s 201ms/step - loss: 0.4698 - accuracy: 0.7811 - val_loss: 0.5672 - val_accuracy: 0.7114
Epoch 5/20
79/79 [==============================] - 16s 196ms/step - loss: 0.4223 - accuracy: 0.8108 - val_loss: 0.4786 - val_accuracy: 0.7640
Epoch 6/20
79/79 [==============================] - 16s 196ms/step - loss: 0.3822 - accuracy: 0.8343 - val_loss: 0.5066 - val_accuracy: 0.7738
Epoch 7/20
79/79 [==============================] - 16s 204ms/step - loss: 0.3533 - accuracy: 0.8489 - val_loss: 0.4642 - val_accuracy: 0.7868

In [45]:
y_pred = model.predict(x_test)

782/782 [==============================] - 7s 8ms/step


In [46]:
y_pred.shape

(25000, 1)

In [47]:
y_test.shape

(25000,)

In [48]:
y_pred_p = (y_pred > 0.5).astype('int64')

In [49]:
print(classification_report(y_test, y_pred_p))

              precision    recall  f1-score   support

           0       0.85      0.71      0.78     12500
           1       0.75      0.88      0.81     12500

    accuracy                           0.80     25000
   macro avg       0.80      0.80      0.79     25000
weighted avg       0.80      0.80      0.79     25000

